# Import

In [ ]:

import numpy as np
import json
from scipy.sparse import load_npz,save_npz,diags,csr_matrix
import scipy.sparse as sp
import pandas as pd
import os
import requests
from io import BytesIO
from tqdm import tqdm
from scipy.sparse.linalg import eigsh
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from pathlib import Path
from matplotlib.backends.backend_pdf import PdfPages
from pypdf import PdfReader, PdfWriter
from tempfile import NamedTemporaryFile
import networkx as nx
import pickle
import gseapy as gp
import mygene
from IPython.display import display, HTML
import re
from collections import deque
from goatools.obo_parser import GODag
import math
from itertools import combinations
from collections import Counter
from gseapy.parser import read_gmt
import time

In [ ]:
pd.set_option('display.width', None)      # No line-wrapping
pd.set_option('display.max_columns', None)  # Show all columns

# Prep

In [ ]:
DISEASE = "BIPOLAR"
DISEASE_FOLDER = f"../output/{DISEASE}/"
RESULT_FOLDER = DISEASE_FOLDER + "leiden_results/"
DGIDB_DIRECTORY = f"../../Gen_Hypergraph/output/DGIDB_{DISEASE}/"
MSIGDB_DIRECTORY = "../../Gen_Hypergraph/output/MSigDB_Full/"
RESULT_COMMUNITIES = "result_communities_new"
RESULT_GRAPH = "result_graph_new"

with open(DISEASE_FOLDER + "gene_to_index_distinct.json", "r") as file:
    gene_to_index_distinct = json.load(file)
with open(DGIDB_DIRECTORY + "gene_to_index_BIPOLAR.json", "r") as file:
    DGIDB_gene_to_index = json.load(file)

In [ ]:
index_to_gene_distinct = {v: k for k, v in gene_to_index_distinct.items()}

In [ ]:
# Community preprocessing
def zscore(values):
    arr = np.asarray(values, dtype=float)
    if arr.size == 0:
        return arr

    mean = arr.mean()
    std = arr.std(ddof=0)

    if std == 0 or np.isnan(std):
        # no variation: all z-scores = 0
        return np.zeros_like(arr)

    return (arr - mean) / std

def communities_cutoff(communities, cutoff = 100):
    result = []
    for community in communities:
        if len(community) >= cutoff:
            result.append(community)

    return result, len(result)

def community_central_genes_by_num(G, community_nodes, weight="weight", top_n=20):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    zscore_list = zscore(ks)
    Z = dict(zip(H,zscore_list))        # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    return [u for u in ranked[:top_n]]

def community_central_genes_by_score(G, community_nodes, weight="weight",score_cap = 1):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    zscore_list = zscore(ks)
    Z = dict(zip(H,zscore_list))        # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    return [u for u in ranked if Z[u] >= score_cap]

def community_central_genes_by_pct(G, community_nodes, weight="weight",pct = 0.3):
    C = set(community_nodes)
    H = G.subgraph(C).copy()                       # induced subgraph
    # within-community (weighted) degree
    k = {u: H.degree(u, weight=weight) for u in H}
    ks = np.array(list(k.values()), dtype=float)
    zscore_list = zscore(ks)
    Z = dict(zip(H,zscore_list))        # within-module degree z-score

    # rank by z
    ranked = sorted(H.nodes(), key=lambda u: (Z[u]), reverse=True)
    top = int(len(ranked)*pct)
    return [u for u in ranked[:top]]



In [ ]:
# Helpful functions
def drop_nan_from_communities(communities):
    cleaned_communities = []
    total_dropped = 0

    for i, community in enumerate(communities):
        cleaned = []
        dropped = 0
        for g in community:
            if g is None or (isinstance(g, float) and math.isnan(g)):
                dropped += 1
            else:
                cleaned.append(g)
        cleaned_communities.append(cleaned)
        total_dropped += dropped
        print(f"Community {i}: dropped {dropped} NaN entries")

    print(f"\nTotal dropped across all communities: {total_dropped}")
    return cleaned_communities

def big_objects(n=10, min_mb=1):
    """
    Show the largest objects currently in memory.
    
    Parameters
    ----------
    n : int
        Number of top objects to show.
    min_mb : float
        Minimum size (in MB) to include.
    """
    import sys
    import numpy as np
    import pandas as pd
    import scipy.sparse as sp
    from IPython import get_ipython

    def get_size(obj):
        try:
            if isinstance(obj, np.ndarray):
                return obj.nbytes
            elif isinstance(obj, pd.DataFrame) or isinstance(obj, pd.Series):
                return obj.memory_usage(deep=True).sum()
            elif sp.issparse(obj):
                return (obj.data.nbytes +
                        obj.indptr.nbytes +
                        obj.indices.nbytes)
            else:
                return sys.getsizeof(obj)
        except Exception:
            return 0

    ip = get_ipython()
    if ip is None:
        ns = globals()
    else:
        ns = ip.user_ns

    items = []
    for name, val in ns.items():
        if name.startswith('_'):
            continue  # skip internals
        size = get_size(val)
        if size > min_mb * 1024 ** 2:
            items.append((name, type(val).__name__, size))

    items.sort(key=lambda x: x[2], reverse=True)

    print(f"{'Variable':30s} {'Type':25s} {'Size (MB)':>10s}")
    print("-" * 70)
    for name, t, size in items[:n]:
        print(f"{name:30s} {t:25s} {size / 1024 ** 2:10.2f}")

In [ ]:
# Z-score distribution
def compute_within_community_zscores(G, communities):
    """
    Compute within-community degree z-scores for each node.

    Parameters
    ----------
    G : networkx.Graph
        Graph whose nodes match those in `communities`.
        Uses edge weight from 'weight' attribute if present; otherwise 1.0.
    communities : list of list
        communities[c] is a list of node IDs in community c.

    Returns
    -------
    pandas.DataFrame
        Columns: ['node', 'community', 'k_in', 'zscore']
    """
    # 1. Node -> community map
    node_to_comm = {
        node: cid
        for cid, comm in enumerate(communities)
        for node in comm
    }

    # 2. Initialize within-community degree for every node
    k_in = {node: 0.0 for node in node_to_comm.keys()}

    # 3. Iterate over edges once, add weight if endpoints in same community
    for u, v, data in G.edges(data=True):
        if u not in node_to_comm or v not in node_to_comm:
            continue  # skip nodes not in any community (if that happens)

        cu = node_to_comm[u]
        cv = node_to_comm[v]

        if cu == cv:
            w = data.get("weight", 1.0)
            k_in[u] += w
            k_in[v] += w

    # 4. Build DataFrame
    rows = [
        (node, node_to_comm[node], k_val)
        for node, k_val in k_in.items()
    ]
    df = pd.DataFrame(rows, columns=["node", "community", "k_in"])

    # 5. Compute z-scores per community using the helper
    df["zscore"] = df.groupby("community")["k_in"].transform(zscore)

    return df

def plot_zscore_hist_all(df_z, bins=20, per_row=4, max_communities=None):
    communities = sorted(df_z["community"].unique())
    if max_communities is not None:
        communities = communities[:max_communities]

    n = len(communities)
    nrows = int(np.ceil(n / per_row))
    ncols = per_row

    fig, axes = plt.subplots(nrows, ncols, figsize=(4*ncols, 3*nrows))
    axes = np.array(axes).flatten()

    for ax, cid in zip(axes, communities):
        zvals = df_z.loc[df_z["community"] == cid, "zscore"]
        ax.hist(zvals, bins=bins)
        ax.set_title(f"Community {cid}")
        ax.set_xlabel("z-score")
        ax.set_ylabel("Frequency")

    for ax in axes[len(communities):]:
        ax.axis("off")

    plt.tight_layout()
    plt.show()

In [ ]:
# Degree distribution
def compute_within_community_degrees(G, communities):
    """Compute within-community degrees for each node."""
    node_to_comm = {node: cid for cid, comm in enumerate(communities) for node in comm}
    k_in = {node: 0.0 for node in node_to_comm}

    for u, v, data in G.edges(data=True):
        if u not in node_to_comm or v not in node_to_comm:
            continue
        cu, cv = node_to_comm[u], node_to_comm[v]
        if cu == cv:
            w = data.get("weight", 1.0)
            k_in[u] += w
            k_in[v] += w

    df = pd.DataFrame([(n, node_to_comm[n], d) for n, d in k_in.items()],
                      columns=["node", "community", "k_in"])
    return df


def plot_topk_community_degree_distributions(G, communities, top_k=20, normalize=True, bins=20):
    """
    Plot within-community degree distributions for the top_k largest communities.

    Parameters
    ----------
    G : networkx.Graph
        Input graph.
    communities : list[list]
        List of node lists (each representing a community).
    top_k : int, optional
        Number of largest communities (by size) to plot. Default is 20.
    normalize : bool, optional
        If True, y-axis shows probability instead of count. Default is True.
    bins : int, optional
        Number of bins in each histogram. Default is 20.
    """
    df = compute_within_community_degrees(G, communities)

    # Select top_k largest communities by node count
    comm_sizes = df["community"].value_counts().sort_values(ascending=False)
    top_comms = comm_sizes.index[:top_k]

    # Determine subplot grid (square-ish)
    ncols = 5
    nrows = int(np.ceil(top_k / ncols))
    fig, axes = plt.subplots(nrows, ncols, figsize=(4*ncols, 3*nrows))
    axes = axes.flatten()

    for i, cid in enumerate(top_comms):
        ax = axes[i]
        vals = df.loc[df["community"] == cid, "k_in"].values
        if len(vals) == 0:
            continue

        weights = np.ones_like(vals) / len(vals) if normalize else None
        ax.hist(vals, bins=bins, weights=weights, color="steelblue", alpha=0.7)
        ax.set_title(f"Community {cid} (n={len(vals)})", fontsize=10)
        ax.set_xlabel("$k_{in}$", fontsize=9)
        ax.set_ylabel("Prob." if normalize else "Count", fontsize=9)
        ax.tick_params(axis="both", labelsize=8)

    # Turn off unused axes
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    fig.suptitle(f"Within-community degree distributions (Top {top_k})", fontsize=14)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    plt.show()

In [ ]:
# Loading result graph and communities
with open(f"{RESULT_FOLDER}/{RESULT_COMMUNITIES}.pkl", "rb") as f:
    communities = pickle.load(f)
with open(f"{RESULT_FOLDER}/{RESULT_GRAPH}.pkl", "rb") as f:
    graph = pickle.load(f)

In [ ]:
communities_df = compute_within_community_zscores(graph, communities)
plot_zscore_hist_all(communities_df, bins=20, per_row=4,max_communities=20)

In [ ]:
plot_topk_community_degree_distributions(graph, communities, top_k=20, normalize=True)

In [ ]:
# Checking communities greater than 100
communities_greater_than_cutoff,num_greater_cutoff = communities_cutoff(communities,30)
print(communities_greater_than_cutoff, '\n', num_greater_cutoff)
print(len(communities))

In [ ]:
# # Update communities to include only the important genes, determined by a community-size_cap and percent_cap
# SIZE_CAP = 30
# PCT = 0.3
# communities_selected = []
# for community in communities:
#     if (len(community) >= SIZE_CAP):
#         important_nodes = community_central_genes_by_pct(graph,community,pct = PCT)
#         communities_selected.append(important_nodes)

In [ ]:
# Update communities to include only the important genes, determined by a community-size_cap and z-score_cap
SIZE_CAP = 30
SCORE_CAP = 0.5
communities_selected = []
for community in communities:
    if (len(community) >= SIZE_CAP):
        important_nodes = community_central_genes_by_score(graph,community,score_cap = SCORE_CAP)
        a = len(important_nodes)
        b = len(community)
        print(a,b,a/b)
        communities_selected.append(important_nodes)

In [ ]:
print(communities_selected)
print(len(communities_selected))

In [ ]:
# Filter the empty communities after first filter
i = 0
for i in range(len(communities_selected)):
    cl = len(communities_selected[i])
    if (cl == 0): del(communities_selected[i])

In [ ]:
print(communities_selected)
print(len(communities_selected))

In [ ]:
# Convert index to ncbi
communities_ncbi = [list(map(index_to_gene_distinct.get, c)) for c in communities_selected]
print(communities_ncbi)

In [ ]:
# NCBI to HGNC symbol
COMMUNITIES_HGNC = []
missed = []


for community in communities_ncbi:
    mg = mygene.MyGeneInfo()
    entrez_ids = [str(e) for e in community]

    results = mg.querymany(
        entrez_ids,
        scopes="entrezgene",
        fields="symbol",
        species="human"
    )

    # Build a mapping: input ID -> symbol (or None)
    id_to_symbol = {}
    for r in results:
        q = str(r.get("query"))
        id_to_symbol[q] = r.get("symbol") if not r.get("notfound") else None

    # Preserve original order
    symbols = [id_to_symbol.get(str(e), None) for e in entrez_ids]
    COMMUNITIES_HGNC.append(symbols)


In [ ]:
print(COMMUNITIES_HGNC)

In [ ]:
COMMUNITIES_HGNC = drop_nan_from_communities(COMMUNITIES_HGNC)

# Categoization Prep

### GO-slim

In [ ]:
DATA_DIRECTORY = "../../data"
GO_OBO = f"{DATA_DIRECTORY}/GO/go-basic.obo"            # put the file in your working dir (or give full path)
GOSLIM_OBO = f"{DATA_DIRECTORY}/GO/goslim_generic.obo"  # swap to another slim if you prefer
GOSLIM_PIR_OBO = f"{DATA_DIRECTORY}/GO/goslim_pir.obo"  # swap to another slim if you prefer
GOSLIM_YEAST_OBO = f"{DATA_DIRECTORY}/GO/goslim_yeast.obo"
GOSLIM_AGR_OBO = f"{DATA_DIRECTORY}/GO/goslim_agr.obo"

In [ ]:
# pip install goatools


# GO library
go = GODag(GO_OBO, optional_attrs={'relationship'})

# SLIM libraries
slim = GODag(GOSLIM_OBO)
slim_pir = GODag(GOSLIM_PIR_OBO)
slim_yeast = GODag(GOSLIM_YEAST_OBO)
slim_agr = GODag(GOSLIM_AGR_OBO)

slim_ids = set(slim.keys())
slim_pir_ids = set(slim_pir.keys())
slim_yeast_ids = set(slim_yeast.keys())
slim_agr_ids = set(slim_agr.keys())


slim_names = {sid: slim[sid].name for sid in slim_ids}

In [ ]:
GO_RE = re.compile(r"(GO:\d{7})")

def get_goid(term: str):
    if isinstance(term, str):
        m = GO_RE.search(term)
        if m:
            return m.group(1)
    return None

def map_to_goslim(go_id,slim_ids_param):
    """Return all GO-Slim IDs that are ancestors (or self) of the GO term."""
    if not go_id or go_id not in go:
        return []
    node = go[go_id]
    parents = node.get_all_parents()
    if parents and not isinstance(next(iter(parents)), str):
        anc_ids = {node.id} | {t.id for t in parents}
    else:
        anc_ids = {node.id} | set(parents)
    return sorted(anc_ids & slim_ids_param)

In [ ]:
from collections import deque

def get_go_ancestors_at_depth(go_id, depth, include_relations=("is_a", "part_of")):
    """
    Return the set of GO term IDs that are exactly `depth` steps above `go_id`.

    Parameters
    ----------
    go_id : str
        Starting GO term (e.g., "GO:0051310").
    depth : int
        Exact distance up the DAG to return (1 = immediate parents).
    include_relations : tuple[str]
        Relation types to traverse upward, e.g. ("is_a", "part_of", "regulates", ...).

    Returns
    -------
    set[str]
        GO IDs at exactly `depth` from `go_id`. Empty set if none.
    """
    if depth < 1:
        return set()
    if go_id not in go:
        return []

    # One hop function honoring relation filter
    def parent_ids(term):
        ids = set()
        if "is_a" in include_relations:
            ids.update(p.id for p in term.parents)  # direct is_a/part_of often merged here
        rel = getattr(term, "relationship", {}) or {}
        for r in include_relations:
            ids.update(rel.get(r, []))
        # ensure IDs exist in DAG
        return {pid for pid in ids if pid in go}

    # BFS by levels until we reach the target depth
    level = 0
    frontier = {go_id}
    visited = {go_id}

    while frontier and level < depth:
        next_frontier = set()
        for node in frontier:
            for pid in parent_ids(go[node]):
                if pid not in visited:
                    visited.add(pid)
                    next_frontier.add(pid)
        level += 1
        frontier = next_frontier

    # When loop ends: if level == depth, `frontier` is exactly-depth nodes
    return frontier if level == depth else set()


### KEGG

In [ ]:
def build_kegg_name_to_id(species="hsa"):
    """Map KEGG pathway name -> 'hsaXXXXX' (species-specific)."""
    lines = requests.get(f"https://rest.kegg.jp/list/pathway/{species}").text.strip().splitlines()
    name_to_id = {}
    for ln in lines:
        pid, raw = ln.split("\t")
        pid = pid.replace("path:", "")  # e.g. hsa03010
        # strip " - Homo sapiens (human)" suffix
        name = re.sub(r"\s*-\s*Homo sapiens.*$", "", raw).strip()
        name_to_id[name.lower()] = pid
    return name_to_id

name_to_id = build_kegg_name_to_id("hsa")

In [ ]:
def get_kegg_level2(hsa_id: str) -> str | None:
    """
    Return the KEGG Level 2 category for a pathway like 'hsa03040'.
    Example: get_kegg_level2("hsa03040") -> 'Transcription'
    """
    url = f"http://rest.kegg.jp/get/{hsa_id}"
    try:
        text = requests.get(url, timeout=10).text
    except Exception:
        return None

    for line in text.splitlines():
        if line.startswith("CLASS"):
            # CLASS line looks like: CLASS       Genetic Information Processing; Transcription
            parts = [p.strip() for p in line.split(";", maxsplit=2)]
            if len(parts) >= 2:
                return parts[1]
            elif len(parts) == 1:
                return parts[0].replace("CLASS", "").strip()
    return None

### Reactome

In [ ]:
def build_reactome_level1_map(species="9606"):
    """
    Returns { 'R-HSA-xxxxx': ['Top Level Name', ...], ... } for the given species.
    species can be a taxonomy id ('9606') or a name ('Homo sapiens').
    """
    # ensure spaces are encoded if a name is used
    species_path = species.replace(" ", "+")
    url = f"https://reactome.org/ContentService/data/eventsHierarchy/{species_path}"
    r = requests.get(url, headers={"Accept": "application/json"}, timeout=60)
    r.raise_for_status()
    trees = r.json()  # list of trees, one per TopLevelPathway

    mapping = {}
    def walk(node, top_name):
        st_id = node.get("stId")
        if st_id:
            mapping.setdefault(st_id, set()).add(top_name)
        for child in node.get("children", []):
            walk(child, top_name)

    for top in trees:
        top_name = top["name"]  # level-1 category name
        walk(top, top_name)

    # sets -> sorted lists
    return {k: sorted(v) for k, v in mapping.items()}

# Example:
reactome_level1 = build_reactome_level1_map("9606")
  # -> ['Signal Transduction']

In [ ]:
print(reactome_level1["R-HSA-9007101"])

# Run Enrichment Analysis

In [ ]:
TERM_SCORE_CAP = 0.001
PERCENTAGE = 0.1

### GO

In [ ]:
# GO Analysis; save terms with small size and high p-value
def go_enrichment(communities,
                  term_score_cap,
                  percentage, 
                  slim_ids = slim_yeast_ids):
    important_terms = pd.DataFrame(columns=["Community Index","Community Size","Term", "Overlap", "Adjusted P-value","Category"])
    i = 0
    num_nonzero_communities = 0
    unique_entries = set()
    
 
        
    for community in communities:
        # Gene Ontology enrichment
        enr_go = gp.enrichr(
            gene_list=community,
            gene_sets=['GO_Biological_Process_2023',
                    'GO_Molecular_Function_2023',
                    'GO_Cellular_Component_2023'],
            organism='Human',
            outdir=None # don't write to disk
        )
        go_df = enr_go.results
        

        # Filter by overlap percentage and adjusted p-value
        mask =  (go_df["Adjusted P-value"] < term_score_cap) & (go_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
        filtered = go_df[mask].copy()
        
        # Categorization from GO-Slim
        filtered["GO_ID"] = filtered["Term"].apply(get_goid)
        filtered["Slim_IDs"] = filtered["GO_ID"].apply(lambda id: map_to_goslim(id, slim_ids_param=slim_ids))
        # filtered["Slim_IDs"] = filtered["GO_ID"].apply(lambda id: get_go_ancestors_at_depth(id, depth=2, include_relations=("is_a", "part_of")))
        
        # Get empty count
        empty_count = (filtered["Slim_IDs"].apply(len) == 0).sum()
        
        # Get slim names    
        filtered["Category"] = filtered["Slim_IDs"].apply(lambda ids: [go[i].name for i in ids])
        
        # Get unique slim names
        new_cate  = set().union(*filtered["Category"])
        unique_entries = unique_entries.union(new_cate)
        
        # Sort
        filtered['Overlap (value)'] = filtered['Overlap'].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]))
        filtered = filtered.sort_values(['Overlap (value)'], ascending=False)
        
        # Add results to important terms
        if not filtered.empty:
            # print size of community
            print(f"Size of community: {len(community)}")
            
            # print number of filtered terms
            print(f"Number of filtered terms: {len(filtered)}")
            print(f"Number of unmapped terms: {empty_count}")      
            
            filtered.loc[:, "Community Index"] = i
            filtered.loc[:, "Community Size"] = len(community)
            important_terms = pd.concat([important_terms, filtered], ignore_index=True)
            display(HTML(filtered[["Community Index",'Term','Overlap','Adjusted P-value',"Slim_IDs","Category"]].head(10).to_html(max_cols=None)))
            num_nonzero_communities += 1

        i += 1
    print(f"{num_nonzero_communities} out of {len(communities)} communities had significant GO terms.")
    return important_terms

In [ ]:
go_important_terms = go_enrichment(COMMUNITIES_HGNC,TERM_SCORE_CAP,PERCENTAGE)

In [ ]:
go_important_terms

In [ ]:
# print(f"Unique categories: {unique_entries}")
# print(f"Number of unique categories: {len(unique_entries)}")

### KEGG

In [ ]:
# KEGG
def kegg_enrichment(communities,
                    term_score_cap,
                    percentage):
    important_terms = pd.DataFrame(columns=["Community Index","Community Size","Term", "Overlap", "Adjusted P-value","Category"])
    i = 0
    num_nonzero_communities = 0
    for community in communities:
        enr_path = gp.enrichr(
            gene_list=community,
            gene_sets=['KEGG_2021_Human'],
            organism='Human',
            outdir=None
        )
        KEGG_df = enr_path.results

        # Filter by overlap percentage and adjusted p-value
        mask =  (KEGG_df["Adjusted P-value"] < term_score_cap) & (KEGG_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
        filtered = KEGG_df[mask].copy()
        
        # Categorization from KEGG Level 2
        filtered["KEGG_ID"] = filtered["Term"].str.replace(r"\s*-\s*Homo sapiens.*$", "", regex=True).str.lower().map(name_to_id)
        filtered["Category"] = filtered["KEGG_ID"].map(get_kegg_level2)
        
        # Sort
        filtered['Overlap (value)'] = filtered['Overlap'].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]))
        filtered = filtered.sort_values(['Overlap (value)'], ascending=False)
        
        # Add results to important terms
        if not filtered.empty:
            # print size of community
            print(f"Size of community: {len(community)}")   
            
            # print number of filtered terms
            print(f"Number of filtered terms: {len(filtered)}")
            filtered.loc[:, "Community Index"] = i
            filtered.loc[:, "Community Size"] = len(community)
            important_terms = pd.concat([important_terms, filtered], ignore_index=True)
            
            # show results
            display(HTML(filtered[["Community Index",'Term','Overlap','Adjusted P-value',"KEGG_ID","Category"]].head(10).to_html(max_cols=None)))
            num_nonzero_communities += 1

        i += 1
    print(f"{num_nonzero_communities} out of {len(communities)} communities had significant GO terms.")
    return important_terms

In [ ]:
kegg_important_terms = kegg_enrichment(COMMUNITIES_HGNC,TERM_SCORE_CAP,PERCENTAGE)

In [ ]:
kegg_important_terms

### Reactome

In [ ]:
# Reactome enrichment
def reactome_enrichment(communities,
                        term_score_cap,
                        percentage):
    important_terms = pd.DataFrame(columns=["Community Index","Community Size","Term", "Overlap", "Adjusted P-value","Category"])
    i = 0
    num_nonzero_communities = 0
    for community in communities:
        enr_path = gp.enrichr(
            gene_list=community,
            gene_sets=['Reactome_2022'],
            organism='Human',
            outdir=None
        )
        Reactome_df = enr_path.results

        # Filter by overlap percentage and adjusted p-value
        mask =  (Reactome_df["Adjusted P-value"] < term_score_cap) & (Reactome_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
        filtered = Reactome_df[mask].copy()
        
        # Categorization from Reactome Level 1
        filtered["Category"] = filtered["Term"].str.extract(r"(R-[A-Z]+-\d+)", expand=False).map(reactome_level1)
        
        # Sort
        filtered['Overlap (value)'] = filtered['Overlap'].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]))
        filtered = filtered.sort_values(['Overlap (value)'], ascending=False)
        # Add results to important terms
        
        if not filtered.empty:
            print(f"Size of community: {len(community)}")
            print(f"Number of filtered terms: {len(filtered)}")
            filtered.loc[:, "Community Index"] = i
            filtered.loc[:, "Community Size"] = len(community)
            important_terms = pd.concat([important_terms, filtered], ignore_index=True)
            display(HTML(filtered[["Community Index",'Term','Overlap','Adjusted P-value',"Category"]].head(10).to_html(max_cols=None)))
            num_nonzero_communities += 1

        
        i += 1
    print(f"{num_nonzero_communities} out of {len(communities)} communities had significant GO terms.")
    return important_terms

In [ ]:
reactome_important_terms = reactome_enrichment(COMMUNITIES_HGNC,TERM_SCORE_CAP,PERCENTAGE)

In [ ]:
reactome_important_terms

### Disease Data Sets

In [ ]:
# disease_term_score_cap = 0.001
# disease_percentage = 0.1
# important_diseases = pd.DataFrame(columns=["Community Index","Community Size","Term", "Overlap", "Adjusted P-value"])

In [ ]:
# # Disease-gene enrichment libraries
# disease_sets = [
#     'DisGeNET_2020', # curated gene–disease associations
#     'GWAS_Catalog_2023', # genome-wide association hits
#     'OMIM_Disease', # Mendelian disorders
#     'Jensen_DISEASES' # text-mined associations
# ]

# # # Disease-gene enrichment Analysis; save terms with small size and high p-value
# i = 0
# for community in communities_HGNC:
#     # Gene Ontology enrichment
#     enr_disease = gp.enrichr(
#         gene_list=community,
#         gene_sets=disease_sets,
#         organism='Human',
#         outdir=None # don't write to disk
#     )
#     enr_disease_df = enr_disease.results.sort_values('Adjusted P-value')
#     print(f"Size of community: {len(community)}")

#     mask =  (enr_disease_df["Adjusted P-value"] < disease_term_score_cap) & (enr_disease_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > disease_percentage))
        
#     filtered = enr_disease_df[mask].copy()
#     if not filtered.empty:
#         filtered.loc[:, "Community Index"] = i
#         filtered.loc[:, "Community Size"] = len(community)
#         important_diseases = pd.concat([important_diseases, filtered], ignore_index=True)

#     display(HTML(filtered[['Term','Overlap','Adjusted P-value']].head(10).to_html(max_cols=None)))
#     i += 1

# Important Terms Analysis

### Graph Building

In [ ]:
def comm_similarity_with_term(x,y):
    return 1-(abs(x-y)/max(x,y))

In [ ]:
important_terms = pd.DataFrame(columns=["Community Index","Community Size","Term", "Overlap", "Adjusted P-value","Category"])

In [ ]:
c = [go_important_terms,kegg_important_terms,reactome_important_terms]
important_terms = pd.concat(c, ignore_index=True)

In [ ]:
# important_terms = important_terms.sort_values(by="Overlap (value)",ascending=False)
important_terms = important_terms.sort_values(by="Community Index")
important_terms

In [ ]:
# Community id to size dict
com_id_to_size = {i : len(COMMUNITIES_HGNC[i]) for i in range(len(COMMUNITIES_HGNC))}

In [ ]:
unique_com_id_to_size = important_terms.drop_duplicates(subset="Community Index", keep="first")

In [ ]:
comm_size_dict = dict(zip(unique_com_id_to_size["Community Index"], unique_com_id_to_size["Community Size"]))

In [ ]:
# df must have: "Community Index", "Term", "Overlap (value)"
work = important_terms.loc[:, ["Community Index", "Term", "Overlap (value)","Category"]].copy()
work["Overlap (value)"] = work["Overlap (value)"].astype(float)

# Ensure (community, term) uniqueness
dupes = work.duplicated(subset=["Community Index", "Term"], keep=False)
if dupes.any():
    raise ValueError("Duplicated (Community Index, Term) rows found; ensure uniqueness first.")

# --- Build edge weights AND collect contributing terms per pair ---
edge_weights = {}              # (u, v) -> float
edge_counts  = {}              # (u, v) -> int
edge_terms   = {}              # (u, v) -> list[(term, contrib)]

for term, sub in work.groupby("Term", sort=False):
    comms  = sub["Community Index"].to_numpy()
    scores = sub["Overlap (value)"].to_numpy()
    if len(comms) < 2:
        continue
    for i, j in combinations(range(len(comms)), 2):
        u, v = comms[i], comms[j]
        if u > v: u, v = v, u  # canonical ordering
        contrib = comm_similarity_with_term(scores[i], scores[j])

        edge_weights[(u, v)] = edge_weights.get((u, v), 0.0) + contrib
        edge_counts[(u, v)]  = edge_counts.get((u, v), 0)    + 1
        edge_terms.setdefault((u, v), []).append((term, contrib))

# Sort contributing terms by contribution desc for each edge
for key in edge_terms:
    edge_terms[key].sort(key=lambda t: t[1], reverse=True)

# --- Build edge list DataFrame (optional, useful to inspect) ---
edge_df = pd.DataFrame(
    [(u, v, edge_weights[(u, v)], edge_counts[(u, v)], edge_terms.get((u, v), []))
     for (u, v) in edge_weights.keys()],
    columns=["u", "v", "weight", "shared_terms", "terms_contrib"]
).sort_values(["weight", "shared_terms"], ascending=[False, False]).reset_index(drop=True)

# --- Build NetworkX graph with attributes ---
G = nx.Graph()
G.add_nodes_from(pd.unique(work["Community Index"]))
for _, r in edge_df.iterrows():
    G.add_edge(
        int(r.u), int(r.v),
        weight=float(r.weight),
        shared_terms=int(r.shared_terms),
        terms_contrib=r.terms_contrib  # list of (term, contrib) sorted desc
    )

### Table

In [ ]:
#--------------Table------------------
term_contribs = []

for term, sub in work.groupby("Term", sort=False):
    comms  = sub["Community Index"].to_numpy()
    scores = sub["Overlap (value)"].to_numpy()
    if len(comms) < 2:
        continue
    for i, j in combinations(range(len(comms)), 2):
        u, v = comms[i], comms[j]
        if u > v:
            u, v = v, u
        contrib = comm_similarity_with_term(scores[i], scores[j])
        cat = sub["Category"].iloc[0] if "Category" in sub.columns else None
        term_contribs.append((u, v, term, contrib, cat))

# 2) Build DataFrame
term_df = pd.DataFrame(term_contribs, columns=["u", "v", "Term", "Contribution","Category"])
# 3) Sort and aggregate terms per edge (keep per-term order)
agg_blocks = []
for (u, v), sub in term_df.groupby(["u", "v"]):
    sub_sorted = sub.sort_values("Contribution", ascending=False)
    block = "\n".join(
        [f"  - {t} [{cat}] ({c:.3f})"
        for t, c, cat in zip(sub_sorted["Term"], sub_sorted["Contribution"], sub_sorted["Category"])]
    )
    total = sub_sorted["Contribution"].sum()
    agg_blocks.append({
        "u": u,
        "v": v,
        "Total Weight": total,
        "Terms (by contribution)": block
    })

# 4) Create final block table
block_df = pd.DataFrame(agg_blocks).sort_values("Total Weight", ascending=False).reset_index(drop=True)
block_df.head(30)
# 5) Display nicely
for _, row in block_df.iterrows():
    print(f"Community pair ({row.u}, {row.v}) — Total Weight = {row['Total Weight']:.3f}")
    print(row["Terms (by contribution)"])
    print("-" * 60)

### Visualization

In [ ]:
# ========== Visualization ==========

# 1) Remove isolates
non_isolates = [n for n, d in G.degree() if d > 0]
H = G.subgraph(non_isolates).copy()

# 2) Layout per connected component (circular)
components = [H.subgraph(c).copy() for c in nx.connected_components(H)]
pos = {}
offset_x = 0.0
gap = 4.0      # horizontal gap between components
radius = 1.5   # radius of each component circle

for comp in components:
    sub_pos = nx.circular_layout(comp, scale=radius)
    sub_pos = {n: (x + offset_x, y) for n, (x, y) in sub_pos.items()}
    pos.update(sub_pos)
    offset_x += 2 * radius + gap

# 3) Scale edge widths by weight
weights = np.array([H[u][v]['weight'] for u, v in H.edges()], dtype=float)
wmin, wmax = (weights.min(), weights.max()) if len(weights) else (1.0, 1.0)
widths = 2.5 if wmax == wmin else 1.0 + 6.0 * (weights - wmin) / (wmax - wmin)

# 4) Node size proportional to your actual community size
sizes_raw = np.array([comm_size_dict.get(n, 1) for n in H.nodes()], dtype=float)

# Normalize sizes to a good plotting range
sizes = 200 + 1000 * (sizes_raw - sizes_raw.min()) / (sizes_raw.max() - sizes_raw.min() + 1e-9)

# 5) Edge labels
edge_labels = {(u, v): f"{H[u][v]['weight']:.2f}" for u, v in H.edges()}

# 6) Draw graph
plt.figure(figsize=(8, 5))

nx.draw_networkx_nodes(
    H, pos,
    node_size=sizes,
    node_color="skyblue",
    edgecolors="black",
    linewidths=0.6,
)
nx.draw_networkx_edges(
    H, pos,
    width=widths,
    edge_color="gray",
    alpha=0.9,
)
nx.draw_networkx_labels(
    H, pos,
    font_size=9,
    font_weight="bold",
)

# Compute custom label positions: move labels closer to one endpoint
edge_label_pos = {}
for (u, v) in H.edges():
    x_u, y_u = pos[u]
    x_v, y_v = pos[v]

    t = 0.7  # 0.5 = middle of edge, >0.5 moves toward u, <0.5 toward v
    x_label = t * x_u + (1 - t) * x_v
    y_label = t * y_u + (1 - t) * y_v

    edge_label_pos[(u, v)] = (x_label, y_label)

nx.draw_networkx_edge_labels(
    H,
    pos,
    edge_labels=edge_labels,
    font_size=8,
    rotate=False,
    label_pos=0.7,  # move label away from the center of each edge
    bbox=dict(boxstyle="round,pad=0.2", fc="white", ec="none", alpha=0.7),
)

plt.title("Community Similarity Graph — Node Size ∝ Community Size", fontsize=12)
plt.axis("off")
plt.tight_layout()
plt.show()

# Robustness Analysis

In [ ]:
def run_enrichment_func(community,term_score_cap,percentage):
    # GO df
    enr_go = gp.enrichr(
        gene_list=community,
        gene_sets=['GO_Biological_Process_2023',
                'GO_Molecular_Function_2023',
                'GO_Cellular_Component_2023'],
        organism='Human',
        outdir=None # don't write to disk
    )
    GO_df = enr_go.results
    mask =  (GO_df["Adjusted P-value"] < term_score_cap) & (GO_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
    GO_df = GO_df[mask].copy()   
    
    # KEGG df
    enr_kegg = gp.enrichr(
        gene_list=community,
        gene_sets=['KEGG_2021_Human'],
        organism='Human',
        outdir=None
    )
    KEGG_df = enr_kegg.results
    mask =  (KEGG_df["Adjusted P-value"] < term_score_cap) & (KEGG_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
    KEGG_df = KEGG_df[mask].copy() 
       
    # Reactome df
    enr_reactome = gp.enrichr(
        gene_list=community,
        gene_sets=['Reactome_2022'],
        organism='Human',
        outdir=None
    )
    Reactome_df = enr_reactome.results  
    mask =  (Reactome_df["Adjusted P-value"] < term_score_cap) & (Reactome_df["Overlap"].apply(lambda x: int(x.split("/")[0])/int(x.split("/")[1]) > percentage))
    Reactome_df = Reactome_df[mask].copy()
    
    
    all_df = [GO_df,KEGG_df,Reactome_df]
    # build result df by concatenating
    result = pd.concat(all_df, ignore_index=True)
    return result

In [ ]:
from json import JSONDecodeError

# ---------------- 1) Safe wrapper that calls YOUR enrichr function ----------------
_ENR_CACHE = {}  # key: tuple(sorted(genes)) -> DataFrame (copy)

def run_enrichment_safe(run_enrichment_func, community, retries=5, base_sleep=0.8):
    """
    Calls user's run_enrichment_func(community) with retries + memoization.
    Returns a DataFrame (possibly empty). Never raises JSONDecodeError outward.
    """
    # Ensure we always pass a list of gene symbols (never a bare string)
    genes = np.atleast_1d(np.array(community, dtype=object)).tolist()
    if len(genes) == 0:
        return pd.DataFrame()

    key = tuple(sorted(genes))
    if key in _ENR_CACHE:
        return _ENR_CACHE[key].copy()

    for a in range(retries):
        try:
            df = run_enrichment_func(genes,TERM_SCORE_CAP,PERCENTAGE)
            if df is None:
                # treat as transient failure to trigger retry
                raise RuntimeError("run_enrichment_func returned None")
            _ENR_CACHE[key] = df.copy()
            return df
        except (JSONDecodeError, OSError, RuntimeError, ValueError) as e:
            # Transient errors from HTTP/JSON/file handling inside gseapy
            if a == retries - 1:
                # Give up: return empty so pipeline continues
                return pd.DataFrame()
            time.sleep(base_sleep * (2 ** a) + np.random.rand() * 0.3)

    return pd.DataFrame()

# ---------------- 2) Minimal bootstrap to record robust terms ----------------
def get_robust_terms(communities_HGNC, run_enrichment_func,
                     R=50, leaveout=0.10, recurrence_cutoff=0.70, seed=42):
    """
    Uses YOUR run_enrichment_func(community)->DataFrame (already filtered to significant terms).
    Returns DataFrame with columns: community_id, term, recurrence (and Gene_set if available).
    """
    rng = np.random.default_rng(seed)
    rows = []

    for cid, community in enumerate(communities_HGNC):
        n = len(community)
        if n == 0:
            continue
        drop_k = max(1, int(np.floor(leaveout * n)))
        counts = Counter()

        for _ in range(R):
            # Jackknife subset (ensure not empty)
            keep = np.ones(n, dtype=bool)
            keep[rng.choice(n, size=min(drop_k, n), replace=False)] = False
            sub = np.atleast_1d(np.array(community, dtype=object)[keep]).tolist()
            if len(sub) == 0:
                continue

            df = run_enrichment_safe(run_enrichment_func, sub)
            if df is None or df.empty:
                continue

            # Your function already returns significant terms; just count them.
            # If it includes multiple libraries, preserve Gene_set to disambiguate names.
            if 'Term' not in df.columns:
                continue  # be defensive

            if 'Gene_set' in df.columns:
                terms = (df[['Term', 'Gene_set']]
                         .dropna()
                         .drop_duplicates()
                         .apply(lambda r: f"{r['Term']}|{r['Gene_set']}", axis=1)
                         .tolist())
            else:
                terms = df['Term'].dropna().drop_duplicates().tolist()

            counts.update(terms)

            # tiny pause helps with API rate limits if your func calls Enrichr internally
            time.sleep(0.03)

        # Keep only robust terms
        for t, c in counts.items():
            freq = c / max(R, 1)
            if freq >= recurrence_cutoff:
                if '|' in t:
                    term, gene_set = t.split('|', 1)
                    rows.append({'Community Index': cid, 'Term': term, 'recurrence': freq, 'Gene_set': gene_set})
                else:
                    rows.append({'Community Index': cid, 'Term': t, 'recurrence': freq})

    return (pd.DataFrame(rows)
              .sort_values(['Community Index', 'recurrence'], ascending=[True, False])
              .reset_index(drop=True))

In [ ]:
twr3 = get_robust_terms([COMMUNITIES_HGNC[1]], run_enrichment_func,
                                R=25, leaveout=0.1, recurrence_cutoff=0)

In [ ]:
twr3

In [ ]:
terms_with_recurrence = get_robust_terms(COMMUNITIES_HGNC, run_enrichment_func,
                                R=10, leaveout=0.1, recurrence_cutoff=0)

In [ ]:
terms_with_recurrence

In [ ]:
# rename important terms to match terms_with_recurrence
important_terms = important_terms.rename(columns={'index': 'community_id'})
important_terms = important_terms.rename(columns={'Term': 'term'})

In [ ]:
terms_with_rec_merged = important_terms.merge(
    terms_with_recurrence[['community_id', 'term', 'Gene_set', 'recurrence']],
    on=['community_id', 'term', 'Gene_set'],
    how='left'
)

terms_with_rec_merged['recurrence'] = terms_with_rec_merged['recurrence'].fillna(0.0)

terms_with_rec_merged = terms_with_rec_merged.sort_values(
    ['community_id', 'recurrence'],
    ascending=[True, False]
).reset_index(drop=True)

In [ ]:
terms_with_rec_merged

In [ ]:
community_summary = (
    terms_with_rec_merged
    .groupby("community_id")["recurrence"]
    .agg(mean_recurrence="mean", term_count="count")
    .reset_index()
)

print(community_summary)

In [ ]:
display(HTML(terms_with_recurrence.to_html(max_cols=None)))

# Checks!

In [ ]:
c1 = communities_ncbi[2]
DGIDB_genes_ncbi = list(DGIDB_gene_to_index.keys())
print(c1)
print(DGIDB_genes_ncbi)

In [ ]:
for c in communities_ncbi:
    overlap_count = len(set(c) & set(DGIDB_genes_ncbi))
    print(overlap_count)